In [24]:

import math, re, ccxt
from pprint import pformat

def get_data_for_graph(price1 = "last", price2 = "last"):
    print('data for: ', price1, price2)
    wex = ccxt.wex()
    data_for_graph = {}
    data = wex.fetch_tickers()
    for key in data:
        data_for_graph.update({key: data[key][price1]})
    ticker_list = list(data_for_graph.keys())
    ticker_list2 = []
    for key in ticker_list:
        ticker_list2.append("{1}/{0}".format(*key.split('/'))) #обратные тикеры
    
    for key in range(0, len(ticker_list)):
        #обратные котировки
        data_for_graph.update({ticker_list2[key]: 1 / (data[ticker_list[key]][price2])})
    
    print(pformat(data_for_graph))
    return data_for_graph
  


In [25]:
'''

Создаем граф в виде списка смежности для каждой ноды - тикера криптовалюты, 
котировки переводим в логарифмический вид, чтобы можо было складывать вместо 
умножения
'''


'\n\nСоздаем граф в виде списка смежности для каждой ноды - тикера криптовалюты, \nкотировки переводим в логарифмический вид, чтобы можо было складывать вместо \nумножения\n'

In [16]:
def create_graph(data_for_graph):
    graph = {}
    jsrates = data_for_graph
    
    pattern = re.compile("([A-Z]{3,5})/([A-Z]{3,5})")
    
    try:
        for key in jsrates:
            matches = pattern.match(key)
    
            conversion_rate = math.log(float(jsrates[key]))
            
            from_rate = matches.group(1).encode('ascii', 'ignore')
    
            to_rate = matches.group(2).encode('ascii', 'ignore')
    
            if from_rate != to_rate:
                if from_rate not in graph:
                    graph[from_rate] = {}
                graph[from_rate][to_rate] = float(conversion_rate)
    
    except AttributeError:
        print('Strange rate')
        pass
    
    #print(pformat(graph))
    # воплощение алгоритма отсюда
    #  https://gist.github.com/joninvski/701720
    return graph


In [17]:

'''
Подготавливаем данные по каждой вершине графа 
и ищем отрицательный цикл в полученном графе

'''

In [18]:


# Step 1: For each node prepare the destination and predecessor
def initialize(graph, source):
    d = {}  # Stands for destination
    p = {}  # Stands for predecessor
    for node in graph:
        d[node] = float('Inf')  # We start admiting that the rest of nodes are very very far
        p[node] = None
    d[source] = 0  # For the source we know how to reach
    return d, p


def relax(node, neighbour, graph, d, p):
    # If the distance between the node and the neighbour is lower than the one I have now
    if d[neighbour] > d[node] + graph[node][neighbour]:
        # Record this lower distance
        d[neighbour] = d[node] + graph[node][neighbour]
        p[neighbour] = node


def retrace_negative_loop(p, start):
    arbitrageLoop = [start]
    next_node = start
    while True:
        next_node = p[next_node]
        if next_node not in arbitrageLoop:
            arbitrageLoop.append(next_node)
        else:
            arbitrageLoop.append(next_node)
            arbitrageLoop = arbitrageLoop[arbitrageLoop.index(next_node):]
            return arbitrageLoop


def bellman_ford(graph, source):
    d, p = initialize(graph, source)
    for i in range(len(graph) - 1):  # Run this until is converges
        for u in graph:
            for v in graph[u]:  # For each neighbour of u
                relax(u, v, graph, d, p)  # Lets relax it

    # Step 3: check for negative-weight cycles
    for u in graph:
        for v in graph[u]:
            if d[v] < d[u] + graph[u][v]:
                return (retrace_negative_loop(p, source))
    return None


In [19]:

'''
Применяем алгоритм поиска и выводим ноды, 
входящие в отрицательный цикл, если он существует.
Расчитываем его стоимость с учетом 0.2% комиссии
'''

In [26]:
 paths = []
 
 data_for_graph = get_data_for_graph()  
 graph = create_graph(data_for_graph)
 for key in graph:
    
    path = bellman_ford(graph, key)

    if path not in paths and not None and len(path) > 3:
        paths.append(path)

for path in paths:
    if path == None:
        print("No opportunity here :(")
    else:
        print(len(path))
        money = 100
        print("Starting with %(money)i in %(currency)s" % {"money": money, "currency": path[0]})           

        for i, value in enumerate(path):
            if i + 1 < len(path):
                start = path[i]
                end = path[i + 1]
                rate = math.exp(graph[start][end])
                money *= rate*0.998
                print("%(start)s to %(end)s at %(rate)f = %(money)f" % {"start": start, "end": end, "rate": rate,
                                                                        "money": money})
print ("\n")

 

data for:  last last


{'BCH/BCHET': 1.4903129657228018,
 'BCH/BTC': 0.0972,
 'BCH/DASH': 2.182,
 'BCH/ETH': 1.741,
 'BCH/EUR': 535.881,
 'BCH/LTC': 5.706,
 'BCH/RUR': 37334.265,
 'BCH/USD': 657.358,
 'BCH/ZEC': 3.579,
 'BCHET/BCH': 0.671,
 'BTC/BCH': 10.2880658436214,
 'BTC/BTCET': 1.5313935681470137,
 'BTC/DASH': 22.431583669807086,
 'BTC/ETH': 17.979144192736424,
 'BTC/EUR': 5523.75756,
 'BTC/LTC': 58.99705014749263,
 'BTC/NMC': 4761.9047619047615,
 'BTC/NVC': 2083.3333333333335,
 'BTC/PPC': 4347.826086956522,
 'BTC/RUR': 385415.20158,
 'BTC/USD': 6780.002,
 'BTC/ZEC': 37.03703703703704,
 'BTCET/BTC': 0.653,
 'DASH/BCH': 0.4582951420714941,
 'DASH/BTC': 0.04458,
 'DASH/DSHET': 1.3458950201884252,
 'DASH/ETH': 0.799,
 'DASH/EUR': 246.013,
 'DASH/LTC': 2.623,
 'DASH/RUR': 17124.666,
 'DASH/USD': 301.77457,
 'DASH/ZEC': 1.642,
 'DSHET/DASH': 0.743,
 'ETH/BCH': 0.5743825387708213,
 'ETH/BTC': 0.05562,
 'ETH/DASH': 1.2515644555694618,
 'ETH/ETHET': 1.6611295681063123,
 'ETH/EUR': 308.19463,
 'ETH/LTC': 3.27034

In [28]:
'''
Для bid/ask

'''


data_for_graph = get_data_for_graph(price1='bid', price2='ask')
graph = create_graph(data_for_graph)
for key in graph:
    
    path = bellman_ford(graph, key)

    if path not in paths and not None and len(path) > 3:
        paths.append(path)

for path in paths:
    if path == None:
        print("No opportunity here :(")
    else:
        print(len(path))
        money = 100
        print("Starting with %(money)i in %(currency)s" % {"money": money, "currency": path[0]})           

        for i, value in enumerate(path):
            if i + 1 < len(path):
                start = path[i]
                end = path[i + 1]
                rate = math.exp(graph[start][end])
                money *= rate*0.998
                print("%(start)s to %(end)s at %(rate)f = %(money)f" % {"start": start, "end": end, "rate": rate,
                                                                        "money": money})
print ("\n")



data for:  bid ask


{'BCH/BCHET': 1.510574018126888,
 'BCH/BTC': 0.0967,
 'BCH/DASH': 2.168,
 'BCH/ETH': 1.733,
 'BCH/EUR': 535.292,
 'BCH/LTC': 5.689,
 'BCH/RUR': 37262.675,
 'BCH/USD': 653.033,
 'BCH/ZEC': 3.567,
 'BCHET/BCH': 0.66,
 'BTC/BCH': 10.309278350515463,
 'BTC/BTCET': 1.524390243902439,
 'BTC/DASH': 22.39641657334826,
 'BTC/ETH': 17.90189760114572,
 'BTC/EUR': 5507.13,
 'BTC/LTC': 58.8235294117647,
 'BTC/NMC': 4545.454545454545,
 'BTC/NVC': 2083.3333333333335,
 'BTC/PPC': 4347.826086956522,
 'BTC/RUR': 383800.8964,
 'BTC/USD': 6774.424,
 'BTC/ZEC': 36.76470588235294,
 'BTCET/BTC': 0.652,
 'DASH/BCH': 0.4582951420714941,
 'DASH/BTC': 0.0445,
 'DASH/DSHET': 1.3458950201884252,
 'DASH/ETH': 0.796,
 'DASH/EUR': 245.105,
 'DASH/LTC': 2.615,
 'DASH/RUR': 17011.146,
 'DASH/USD': 301.12782,
 'DASH/ZEC': 1.635,
 'DSHET/DASH': 0.738,
 'ETH/BCH': 0.573394495412844,
 'ETH/BTC': 0.05557,
 'ETH/DASH': 1.2468827930174562,
 'ETH/ETHET': 1.6556291390728477,
 'ETH/EUR': 307.25515,
 'ETH/LTC': 3.27034,
 'ETH/RUR

In [29]:


data_for_graph = get_data_for_graph(price1='ask', price2='bid')
graph = create_graph(data_for_graph)
for key in graph:
    
    path = bellman_ford(graph, key)

    if path not in paths and not None and len(path) > 3:
        paths.append(path)

for path in paths:
    if path == None:
        print("No opportunity here :(")
    else:
        print(len(path))
        money = 100
        print("Starting with %(money)i in %(currency)s" % {"money": money, "currency": path[0]})           

        for i, value in enumerate(path):
            if i + 1 < len(path):
                start = path[i]
                end = path[i + 1]
                rate = math.exp(graph[start][end])
                money *= rate*0.998
                print("%(start)s to %(end)s at %(rate)f = %(money)f" % {"start": start, "end": end, "rate": rate,
                                                                        "money": money})
print ("\n")




data for:  ask bid


{'BCH/BCHET': 1.5151515151515151,
 'BCH/BTC': 0.097,
 'BCH/DASH': 2.182,
 'BCH/ETH': 1.743,
 'BCH/EUR': 539.045,
 'BCH/LTC': 5.723,
 'BCH/RUR': 37450.091,
 'BCH/USD': 657.686,
 'BCH/ZEC': 3.591,
 'BCHET/BCH': 0.662,
 'BTC/BCH': 10.34126163391934,
 'BTC/BTCET': 1.5337423312883436,
 'BTC/DASH': 22.471910112359552,
 'BTC/ETH': 17.995321216483713,
 'BTC/EUR': 5540.38513,
 'BTC/LTC': 58.99705014749263,
 'BTC/NMC': 4761.9047619047615,
 'BTC/NVC': 2127.6595744680853,
 'BTC/PPC': 4545.454545454545,
 'BTC/RUR': 386730.40316,
 'BTC/USD': 6774.424,
 'BTC/ZEC': 37.03703703703704,
 'BTCET/BTC': 0.656,
 'DASH/BCH': 0.46125461254612543,
 'DASH/BTC': 0.04465,
 'DASH/DSHET': 1.3550135501355014,
 'DASH/ETH': 0.802,
 'DASH/EUR': 246.922,
 'DASH/LTC': 2.63,
 'DASH/RUR': 17142.618,
 'DASH/USD': 303.05342,
 'DASH/ZEC': 1.644,
 'DSHET/DASH': 0.743,
 'ETH/BCH': 0.5770340450086555,
 'ETH/BTC': 0.05592,
 'ETH/DASH': 1.2562814070351758,
 'ETH/ETHET': 1.6611295681063123,
 'ETH/EUR': 309.13411,
 'ETH/LTC': 3.28859

KeyError: None